# **Demo - Interfaccia I2C**

Il notebook mostra come usare $\texttt{Analog Discovery 2}$ per instaurare una connessione seriale di tipo $\texttt{I2C}$.

In questo caso lo script comunica con il sensore $\texttt{MPU-6050}$ e utilizza i seguenti registri:

* `0x6B` Power management
* `0x41` Registro temperatura H
* `0x42` Registro temperatura L

**Interpretazione della lettura**. Il sensore risponde con 6 bytes, i primi 2 bytes codificano la temperatura, nel datasheet è riportato il significato degli altri 4. I due bytes letti vengono entrambe codificati come un `uint16` che chiameremo `utemp`. La conversione finale richiede

        temp = utemp / 340 + 36.53

**Lettura dell'accelerazione/rotazioni'**. può essere implementata in modo analogo, cercando il registro giusto e la conversione corretta

**Qualche accortezza**. Ricordiamo che il sensore fornisce i necessari {\em pull-up} per il corretto funzionamento dell'interfaccia I2C, quindi è fondamentale che il sensore sia alimentato altrimenti l'avvio del bus I2C darà un errore.

In [4]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time

# -[MPU6050 - Qualche comando]---------------------------------------------------
PWR_MGMT1 = 0x6B
TEMP_OUT_H = [0x41]
TEMP_OUT_L = [0x42]
SAD = 0x68

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
ad2.vdd = 3.3
ad2.power(True)
#   3. Impostazione bus I2C
i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
devs = i2c.scan()  # verifica cosa è connesso...
for dev in devs:
    print(f"Device: 0x{dev:02x}")
#   4. Apre la comunicazione I2C come Master
sht = tdwf.I2Cdevice(ad2.hdwf,SAD)

# -[Configurazione sensore]------------------------------------------------------
sht.write([PWR_MGMT1, 0x00])  

# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_key(event):
    global flag_run
    if event.key == 'escape':  # termina programma
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, ax = plt.subplots(figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect("key_press_event", on_key)

#   2. Inizializzazione variabili
mytime = []
TTv = []
flag_first = True
flag_run = True

start_time = time.time()

#   3. Ciclo di misura
while flag_run:
    time.sleep(0.1)
    sht.writeread(TEMP_OUT_H,1)
    TH = sht.vals[0]
    sht.writeread(TEMP_OUT_L,1)
    TL = sht.vals[0]

     # Complemento a due
    TT = (TH *256.0 ) + TL
    if TT>0x8000:
        TT-=0x10000
    TT = TT/340 + 36.53 # Conversione temperatura
    TTv.append(TT)
    print(f"T = {TT:.2f}C")

    mytime.append(time.time()-start_time)
    if flag_first:
        flag_first = False
        hp1, = plt.plot(mytime,TTv, "o", color="tab:orange")
        plt.grid(True)
        plt.ylabel("Temperature [C]")
        plt.xlabel("Time [s]")
        plt.title("Press ESC to exit")
        plt.show(block=False)    
        plt.tight_layout()
    else:
        hp1.set_ydata(TTv)
        hp1.set_xdata(mytime)
        ax.relim()           
        ax.autoscale_view() 
        fig.canvas.draw()
        fig.canvas.flush_events()
# ---------------------------------------
plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x68
T = 23.54C
T = 23.64C
T = 23.64C
T = 23.64C
T = 23.64C
T = 23.02C
T = 24.44C
T = 24.34C
T = 22.98C
T = 23.02C
T = 23.73C
T = 22.98C
T = 23.82C
T = 23.82C
T = 23.87C
T = 23.78C
T = 23.78C
T = 23.82C
T = 23.73C
T = 23.92C
T = 23.82C
T = 23.92C
T = 23.73C
T = 23.82C
T = 23.82C
T = 23.82C
T = 23.82C
T = 23.78C
T = 23.92C
T = 23.82C
T = 23.78C
T = 23.87C
T = 23.87C
T = 23.92C
T = 23.82C
T = 23.78C
T = 23.87C
T = 23.87C
T = 23.92C
T = 23.92C
T = 23.87C
T = 23.92C
T = 23.92C
T = 23.82C
T = 23.82C
T = 23.97C
T = 23.92C
T = 23.78C
T = 23.64C
T = 23.45C
T = 23.40C
T = 23.35C
T = 23.26C
T = 23.21C
T = 23.02C
T = 23.21C
T = 23.12C
T = 23.02C
T = 23.02C
T = 23.02C
T = 23.02C
T = 22.98C
T = 23.68C
T = 22.84C
T = 22.93C
T = 22.79C
T = 22.74C
T = 22.84C
T = 22.84C
T = 22.88C
T = 22.88C
T = 22.79C
T = 22.79C
T = 22.69C
T = 22.79C
T = 22.69C
T = 22.79C
T = 22.69C
T = 22.65C
T = 22.69C
T = 22.65C
T = 22.65